<a href="https://colab.research.google.com/github/seongmin0219/Kaggle/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
import os
from pathlib import Path

Mounted at /gdrive


In [2]:
Path('./gdrive/MyDrive/')

PosixPath('gdrive/MyDrive')

In [3]:
file_path = '/gdrive/MyDrive/'

if os.path.exists(file_path):
  print('exists')

exists


In [4]:
# kaggle은 ram이 부족해서 gdrive에 업로드하고 구글 코랩프로에서 실행하기
#!ls /gdrive/MyDrive

In [5]:
import json
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [6]:
import gc
import pandas as pd
import numpy as np
import gc 
from datetime import datetime
import os
import time
import json
# 아이템 연관도를 봐야되는데
import re
from pathlib import Path
import numba as nb
from tqdm import tqdm

In [7]:
DATA_DIR = '/gdrive/MyDrive/data/'

In [8]:
# Opening JSON file
f = open(DATA_DIR + 'train_user_seq_log.json')

# returns JSON object asㄴㄴㄴ
# a dictionary
data = json.load(f)

# Iterating through the jsoa

NameError: ignored

In [9]:
df = pd.DataFrame(list(data.items()),columns = ['user_id','item_timestamp'])

In [10]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamp'] = timestamps
    return train_df

22757it [00:03, 7234.68it/s]


KeyboardInterrupt: ignored

In [11]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)
    return json_obj

In [12]:
train_db = generate_train(df)

672116it [00:42, 15779.87it/s]


In [13]:
train_db = train_db.rename(columns={'item_ids': 'item_id', 'user_ids':'user_id'})

In [14]:
# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게

@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps ):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = 0
    max_idx = len(item_timestamps)
    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
          # can huddle point
            timestamp_i = time.mktime(datetime.strptime(item_timestamps[i][1],"%Y-%m-%d %H:%M:%S").timetuple())
            timestamp_j = time.mktime(datetime.strptime(item_timestamps[j][1],"%Y-%m-%d %H:%M:%S").timetuple())
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: # 두 아이템이 같으면 같은 세션이라도 넘어간다
                continue
            if mode == TIME_WEIGHT:
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
            pairs[(item_timestamps[i][0], item_timestamps[i][0])] = w

In [15]:
# @nb.jit(nopython = True, parallel = True, cache = True)
# def get_pairs(aids, ts, ops, row, cnts, ops_weights, mode):
#     par_n = len(row)
#     pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)]

In [108]:
# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps ):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = 0
    max_idx = len(item_timestamps)
    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w


array([8.05696000e+05, 1.61304862e+09])

In [ ]:
#  par_n = len(row)
#   pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)]
#   for par_i in nb.prange(par_n):
#       _, idx, length, start_time = row[par_i]
#       get_single_pairs(pairs[par_i], aids, ts, ops, idx, length, start_time, ops_weights, mode)

In [186]:
# parallel 도 true 인데 왜이리 느리니
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(idx,item_timestamps, cnts,total_len):
    # pairs를 계속 생산하는 것은 쫌
    pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(total_len)]
    # get_single_pair(pairs[idx], item_timestamps)
    # for (item_id1, item_id2), w in pairs[idx].items():
    #     if item_id1 not in cnts: cnts[item_id1] = {0:0.0 for _ in range(0)}
    #     cnt = cnts[item_id1]
    #     if item_id2 not in cnt: cnt[item_id2] = 0.0 
    #     cnt[item_id2] += w

In [114]:
# parallel 도 true 인데 왜이리 느리니
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(idx,row, cnts,total_len,par_n):
    # pairs를 계속 생산하는 것은 좀 그렇다
    pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(total_len)]
    # for par_i in nb.prange(par_n):
      # row로 넘기면 한번에 [item_id,timestamp]로 넘기는 게 아닌 item_id array 따로, timestamp array 따로 넘기기 
    get_single_pair(pairs[idx], row)
    # for (item_id1, item_id2), w in pairs[idx].items():
    #     if item_id1 not in cnts: cnts[item_id1] = {0:0.0 for _ in range(0)}
    #     cnt = cnts[item_id1]
    #     if item_id2 not in cnt: cnt[item_id2] = 0.0 
    #     cnt[item_id2] += w

In [ ]:
# parallel 도 true 인데 왜이리 느리니
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(idx,row, cnts,total_len,par_n):
    # pairs를 계속 생산하는 것은 좀 그렇다
    pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(total_len)]
    for par_i in nb.prange(par_n):
      # row로 넘기면 한번에 [item_id,timestamp]로 넘기는 게 아닌 item_id array 따로, timestamp array 따로 넘기기 
      get_single_pair(pairs[idx], row)
    # for (item_id1, item_id2), w in pairs[idx].items():
    #     if item_id1 not in cnts: cnts[item_id1] = {0:0.0 for _ in range(0)}
    #     cnt = cnts[item_id1]
    #     if item_id2 not in cnt: cnt[item_id2] = 0.0 
    #     cnt[item_id2] += w

In [115]:
# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps ):
    TIME_WEIGHT =1

    mode = TIME_WEIGHT
    min_idx = 0
    max_idx = len(item_timestamps)
    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w


In [40]:
# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps ):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = 0
    max_idx = len(item_timestamps)
    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w


In [21]:
@nb.jit(nopython=True, cache=True)
def get_topk(cnts, topk , k ):
    for item_id , cnt in cnts.items():
        topk[item_id] = np.array(heap_topk(cnt,1,k))

In [22]:
import heapq

In [23]:
@nb.jit(nopython=True , cache=True)
def heap_topk(cnt, overwrite ,cap):
    q = [(0.0, 0, 0) for _ in range(0)]
    for i, (k,n) in enumerate(cnt.items()):
        if overwrite == 1:
            heapq.heappush(q, (n,i,k))
        else:
            heapq.heappush(q, (n,-i,k))
        if len(q) > cap:
            heapq.heappop(q)
    return [heapq.heappop(q)[2] for _ in range(len(q))][::-1]

In [24]:
topk = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.types.int64[:])
cnt = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
cnts = nb.typed.Dict.empty(
    key_type=nb.types.int64,
    value_type=nb.typeof(nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)))

In [ ]:
# 유저 단위로 끊기


In [25]:
# tail = 30
# min_idx = 0
# # 5개 전까지 확인
# length =20
# idx = 0
# TIME_WEIGHT =1
# topn = 20
# # for start_idx in range(len(grp_df)-5):
# n = 200000  #chunk row size
# list_df = [train_db[i:i+n] for i in range(0,train_db.shape[0],n)]
# chunks_1 = list_df[0]
# grp_chunk = chunks_1.groupby('user_id')
# # chunk에서 user_id가 짤렸다면 unique user id 보다 길이가 더 늘어날 수 있으므로 len(df)+알파
# # 161개의 청크이므로 중복은 161개 정도에서만 발생, 안 발생하도록 코딩하는 게 제일 좋지만
# pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(len(df)+300)]
# total_len = len(df)
# for chunk, each_df in  enumerate(list_df):
#     grp_chunk = each_df.groupby('user_id')
#     for idx,grp in tqdm(enumerate(grp_chunk)):
#         grp_df = grp[1]
#         # huddle point data preprocessing , logic 구현 
#         grp_df['mktimestamp'] = grp_df['timestamp'].apply(lambda x: time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()))
#         # grp_df['user_id_typing'] = grp_df['user_id'].apply(lambda x: np.int32(x[-1]))
#         grp_df['item_id_typing'] = grp_df['item_id'].apply(lambda x: np.int32(x.split("_")[-1]))

#         item_timestamps = grp_df[['item_id_typing','mktimestamp']].values
#         get_pairs(idx ,item_timestamps ,cnts,total_len)
#     if chunk ==1:
#       break


In [24]:
#df['mktimestamp'] = df['item_timestamp'].apply(lambda x: time.mktime(datetime.strptime(x[0][1],"%Y-%m-%d %H:%M:%S").timetuple()))

In [25]:
#df['item_timestamps_2'] = df['item_timestamp'].apply(lambda x: np.array([np.int32(x[0][0].split("_")[-1]) , time.mktime(datetime.strptime(x[0][1],"%Y-%m-%d %H:%M:%S").timetuple())]) )

In [50]:
#df['item_timestamps_2'] = df['item_timestamp'].apply(lambda x: np.array([np.int32(x[0][0].split("_")[-1]) , time.mktime(datetime.strptime(x[0][1],"%Y-%m-%d %H:%M:%S").timetuple())]) )

In [62]:
df['new_item_id'] = df['item_timestamp'].apply(lambda x: np.int32(x[0][0].split("_")[-1]) )

In [63]:
 
df['new_timestamp'] = df['item_timestamp'].apply(lambda x: time.mktime(datetime.strptime(x[0][1],"%Y-%m-%d %H:%M:%S").timetuple()) )

In [193]:
train_db['item_id'] = train_db['item_id'].apply(lambda x: np.int32(x.split("_")[-1]) )
train_db['timestamp'] = train_db['timestamp'].apply(lambda x: time.mktime(datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timetuple()) )

In [194]:
items_timestamps = train_db[['item_id','timestamp']].values

In [190]:
# array 의 값의 type은 같아야 유지된다 

In [ ]:
df['item_id'].apply(lambda x: np.int32(x.split("_")[-1]))

In [140]:
#np.concatenate((df['new_item_id'].values[0],df['new_timestamp'].values[0] ),axis=0).shape

(3428,)

In [ ]:
#  Failed in nopython mode pipeline (step: nopython frontend)
# non-precise type array(pyobject, 2d, F)
# During: typing of argument at <ipython-input-31-aead97c69ce9> (4)

In [28]:
# 이전에 됐던 거 자체가 문제, 하나밖에 안된 거 기 때문에
df['new_item_id'] = df['item_timestamp'].apply(lambda x: np.array([ np.int32(val[0].split("_")[-1]) for val in x])  )
df['new_timestamp'] = df['item_timestamp'].apply(lambda x: np.array([ time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple()) for val in x ]) )

array([[array([805696, 386903,   3832, ..., 784458,   6821, 944280], dtype=int32),
        array([1.61304862e+09, 1.61304863e+09, 1.61304865e+09, ...,
               1.62057312e+09, 1.62057317e+09, 1.62060035e+09])    ],
       [array([ 97057, 644971, 947129,  58840, 640213, 954408, 485937, 817894,
               281063, 200041, 831874, 689622, 729188, 332555, 148905, 138672,
               521110, 277200, 895414, 767092, 789115, 917773, 908817, 156502,
               176791, 844058, 772662, 752807, 470257, 616117, 562328, 667821,
               201524, 687037, 757166, 138672, 493181, 338910, 440210, 146985,
               502852, 670721, 775310, 207325, 749595, 625510, 207325, 597318,
               720637, 954267,  99569,  14460, 294502, 225432,  61544, 922718,
               423536, 654452, 467663, 237931, 348535,  89776, 662118, 652918,
               728229, 293913, 309024, 652918, 699487, 838574, 448649, 122877,
               430325, 120567, 376167, 687037, 693653, 438077, 35974

In [34]:
len(temp[:par_n])

1714

In [36]:

temp[:par_n]

array([[8.05696000e+05, 1.61304862e+09],
       [3.86903000e+05, 1.61304863e+09],
       [3.83200000e+03, 1.61304865e+09],
       ...,
       [7.84458000e+05, 1.62057312e+09],
       [6.82100000e+03, 1.62057317e+09],
       [9.44280000e+05, 1.62060035e+09]])

In [47]:

len(temp[idx])

IndexError: ignored

In [52]:

len(temp[1])

23

In [48]:

idx

1024

In [56]:
min(idx+parallel,min_idx)

0

In [61]:
len(row)

1024

In [64]:
len(temp)

382986

In [67]:

len(row)

1024

In [72]:
len(df.iloc[0:1024][['new_item_id','new_timestamp']].values)

1024

In [74]:
len(row)

1024

In [76]:

len(temp)

1024

In [82]:
len(temp)

1024

In [83]:
idx

1024

In [91]:
len(row[0])

2

In [93]:
t = np.array(list(map(lambda x : np.concatenate((np.reshape(x[0], newshape=(-1,1)) , np.reshape(x[1], newshape=(-1,1)) ),axis=1),row)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [97]:
len(input)

382986

In [98]:
input[:5]

array([[8.05696000e+05, 1.61304862e+09],
       [3.86903000e+05, 1.61304863e+09],
       [3.83200000e+03, 1.61304865e+09],
       [8.49824000e+05, 1.61304870e+09],
       [8.15594000e+05, 1.61304878e+09]])

In [102]:

len(temp[0])

1714

In [103]:
a= 3
c = a


In [104]:
a = 5

In [105]:
c

3

In [135]:

len(row)

10

In [142]:
row = df.iloc[idx:min(idx + parallel, max_idx)][['session', 'idx']]

KeyError: ignored

In [ ]:
# aid array에서 idx로 접근해서 꺼내 쓸 수 있도록 해보자 그게 cs적으로 빠르다면
# aids: aid array
# ts: time stamp array (minused by start_time of corresponding session)
# ops: type array, clicks=0, carts=1, orders=2

NameError: ignored

In [151]:
df

,user_id,item_timestamp,new_item_id,new_timestamp,length
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268
...,...,...,...,...,...
672111,user_829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8
672112,user_829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2
672113,user_829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2
672114,user_829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2


In [157]:
row = df.iloc[idx:min(idx + parallel, max_idx)][['user_id', 'idx', 'length']].values

In [158]:
row


array([['user_0', 0, 1714],
       ['user_1', 1, 197],
       ['user_2', 2, 9296],
       ...,
       ['user_1061', 1021, 5],
       ['user_1063', 1022, 321],
       ['user_1064', 1023, 8]], dtype=object)

In [154]:
df['idx']  = df.index.tolist()

In [ ]:
get_pairs(aids, ts, ops, row, cnts, ops_weights, mode)

In [162]:
df.iloc[0]['new_item_id'][-1]

944280

In [161]:
items_timestamps[:1714]

array([['item_805696', '2021-02-11 13:03:42'],
       ['item_386903', '2021-02-11 13:03:52'],
       ['item_3832', '2021-02-11 13:04:07'],
       ...,
       ['item_784458', '2021-05-09 15:11:59'],
       ['item_6821', '2021-05-09 15:12:50'],
       ['item_944280', '2021-05-09 22:45:46']], dtype=object)

In [163]:
row = df.iloc[idx:min(idx + parallel, max_idx)][['user_id', 'idx', 'length']].values

,user_id,item_timestamp,new_item_id,new_timestamp,length,idx
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714,0
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197,1
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296,2
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52,3
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268,4
...,...,...,...,...,...,...
672111,user_829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8,672111
672112,user_829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2,672112
672113,user_829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2,672113
672114,user_829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2,672114


In [175]:
df['idx'] = df['length'].cumsum()

In [177]:

train_db

,user_id,item_id,timestamp
0,user_0,item_805696,2021-02-11 13:03:42
1,user_0,item_386903,2021-02-11 13:03:52
2,user_0,item_3832,2021-02-11 13:04:07
3,user_0,item_849824,2021-02-11 13:05:04
4,user_0,item_815594,2021-02-11 13:06:19
...,...,...,...
32134194,user_829407,item_183816,2021-05-12 13:00:47
32134195,user_829408,item_485803,2021-05-12 13:00:42
32134196,user_829408,item_1047,2021-05-12 13:01:57
32134197,user_829414,item_710805,2021-05-12 13:02:31


In [179]:
df['idx'] = df['idx'].shift(1)

In [181]:
df.iloc[0]['idx'] = 0


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [183]:

df.fillna(0,inplace=True)

In [186]:
df['idx'] = df['idx'].astype(np.int64)


array(['item_97057', '2021-02-11 13:03:42'], dtype=object)

In [196]:
df

,user_id,item_timestamp,new_item_id,new_timestamp,length,idx
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714,0
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197,1714
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296,1911
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52,11207
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268,11259
...,...,...,...,...,...,...
672111,user_829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8,32134183
672112,user_829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2,32134191
672113,user_829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2,32134193
672114,user_829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2,32134195


In [ ]:
_, idx, length = row[par_i]

In [225]:


@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(items_timestamps,row, cnts,par_n):
    pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)] 

    for par_i in nb.prange(par_n):
      # if par_i % 100 ==0:
        # print(par_i)
      user_id,idx, length = row[par_i]
      get_single_pair(pairs[par_i], items_timestamps ,idx, length)
      
    for par_i in range(par_n):
        for (aid1, aid2), w in pairs[par_i].items():
            if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
            cnt = cnts[aid1]
            if aid2 not in cnt: cnt[aid2] = 0.0
            cnt[aid2] += w

    # for par_i in range(par_n):
    #     for (aid1, aid2), w in pairs[par_i].items():
    #         if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
    #         cnt = cnts[aid1]
    #         if aid2 not in cnt: cnt[aid2] = 0.0
    #         cnt[aid2] += w

# 어떻게 끊느냐도 중요한데 유저 전체를 보고 timestamp 로 끊는다?
# 시퀀스 학습시키는게
@nb.jit(nopython=True , cache=True)
def get_single_pair(pairs, item_timestamps,idx,length):
    TIME_WEIGHT =1
    mode = TIME_WEIGHT
    min_idx = idx
    max_idx = idx+length

    for i in range(min_idx , max_idx):
        for j in range(i+1, max_idx):
            timestamp_i =  item_timestamps[i][1]
            timestamp_j =  item_timestamps[j][1]
            # 하루가 넘어가면 break
            time_diff = timestamp_j - timestamp_i
            if time_diff  >= 24 * 60 * 60: break
            if item_timestamps[i][0] == item_timestamps[j][0]: continue # 두 아이템이 같으면 같은 세션이라도 넘어간다
            # 가까운 시간이라도 전혀 다른 아이템일 수 있다
            if mode == TIME_WEIGHT:
                if 0 <= time_diff <= 300: w = 5
                elif 300< time_diff <= 600: w = 4
                elif 600< time_diff <=1800: w = 3
                elif 1800 < time_diff <=3600: w = 2
                elif 3600 < time_diff <= 3600 * 6: w = 1
                else: w = 0
            pairs[(item_timestamps[i][0], item_timestamps[j][0])] = w



In [207]:
row

array([['user_0', 0, 1714],
       ['user_1', 1714, 197],
       ['user_2', 1911, 9296],
       ...,
       ['user_1061', 382652, 5],
       ['user_1063', 382657, 321],
       ['user_1064', 382978, 8]], dtype=object)

In [209]:
df['user_id'] = df['user_id'].apply(lambda x : np.int32(x.split('_')[-1]))

In [210]:
df


,user_id,item_timestamp,new_item_id,new_timestamp,length,idx
0,0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714,0
1,1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197,1714
2,2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296,1911
3,3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52,11207
4,4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268,11259
...,...,...,...,...,...,...
672111,829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8,32134183
672112,829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2,32134191
672113,829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2,32134193
672114,829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2,32134195


In [226]:
%%time
# 30초에 1프로
# 3000 초 100프로 50분? 갈수록 빨라지는 듯 cache 해서 그런지  1분에 65건 정도니까원래 코드랑비슷하다 내가 이전에 짠거랑 비교하기 작은 차이라도 큰 차이난다 속도가 엄청나게 !!

# cnt저장이 오래걸린다 그래도 할만한 속도로 대폭 줄음
# 빠르고 병렬적인 base line end-to-end를 위ㅎ 
tail = 30
min_idx = 0
# 5개 전까지 확인
length =20
idx = 0
TIME_WEIGHT =1
parallel =1024
topn = 20
# for start_idx in range(len(grp_df)-5):
n = 200000  #chunk row size
list_df = [train_db[i:i+n] for i in range(0,train_db.shape[0],n)]
chunks_1 = list_df[0]
grp_chunk = chunks_1.groupby('user_id')
# chunk에서 user_id가 짤렸다면 unique user id 보다 길이가 더 늘어날 수 있으므로 len(df)+알파
# 161개의 청크이므로 중복은 161개 정도에서만 발생, 안 발생하도록 코딩하는 게 제일 좋지만
pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(len(df)+300)]
total_len = len(df) 
max_idx = len(df)
cnts = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
# 확연히 다른 속도.. 일단은 자자 60만행에대해서 각 행 처리한듯하니 옛날같았으면 느리네하고 기다릴텐데

for idx in tqdm(range(0,max_idx, parallel)):
    row = df.iloc[idx:min(idx + parallel, max_idx)][['user_id', 'idx', 'length']].values
    par_n = len(row)
    get_pairs(items_timestamps, row, cnts,par_n)

 

100%|██████████| 657/657 [03:15<00:00,  3.36it/s]

CPU times: user 4min 38s, sys: 3.01 s, total: 4min 41s
Wall time: 3min 15s


In [227]:

df

,user_id,item_timestamp,new_item_id,new_timestamp,length,idx
0,0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714,0
1,1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197,1714
2,2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296,1911
3,3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52,11207
4,4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268,11259
...,...,...,...,...,...,...
672111,829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8,32134183
672112,829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2,32134191
672113,829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2,32134193
672114,829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2,32134195


In [229]:
# 한 아이템에 이 데이터들 저장하는게 5분이면 아주 빠르다 굿 안알려진거라 공유 받은 이거 나만 잘쓰고 있는 걸 수도 드디어 해냈따 자료구조는 컴퓨터과학적으로 생각하고 꼼꼼함을 곁들여서 정성적으로
cnts[710805]

DictType[int64,float64]<iv=None>({205108: 974.0, 911302: 24.0, 27526: 434.0, 561965: 24.0, 716642: 66.0, 247703: 70.0, 180739: 40.0, 775306: 99.0, 852611: 108.0, 267761: 19.0, 753713: 506.0, 35296: 1097.0, 310343: 214.0, 487164: 21.0, 129407: 47.0, 102468: 65.0, 883209: 31.0, 556108: 2.0, 153293: 25.0, 399654: 501.0, 569886: 233.0, 146176: 67.0, 764004: 1382.0, 885499: 1456.0, 396738: 458.0, 319370: 433.0, 490906: 3318.0, 424308: 2440.0, 204302: 54.0, 344325: 5.0, 163988: 713.0, 31473: 284.0, 385955: 183.0, 720743: 461.0, 114789: 1284.0, 268392: 36.0, 364964: 56.0, 383142: 10.0, 731240: 6.0, 23674: 68.0, 819060: 62.0, 229779: 58.0, 53796: 47.0, 26553: 24.0, 381746: 24.0, 108179: 13.0, 366524: 34.0, 426840: 201.0, 825840: 142.0, 19529: 349.0, 466540: 125.0, 716120: 8.0, 743305: 145.0, 931859: 30.0, 379100: 13.0, 910858: 12.0, 35791: 4.0, 810648: 2.0, 363368: 170.0, 199691: 53.0, 194914: 423.0, 88696: 104.0, 479021: 49.0, 371235: 578.0, 416605: 65.0, 298705: 35.0, 32355: 76.0, 937224: 12

In [222]:
pairs[0]

{}

In [220]:

cnts.keys()

KeysView(DictType[int64,DictType[int64,float64]<iv=None>]<iv=None>({}))

In [146]:
items_timestamps = train_db[['item_id','timestamp']].values

In [147]:
items_timestamps[0]

array(['item_805696', '2021-02-11 13:03:42'], dtype=object)

In [149]:
df['length'] = df['item_timestamp'].apply(lambda x : len(x))

,user_id,item_timestamp,new_item_id,new_timestamp,length
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386...","[805696, 386903, 3832, 849824, 815594, 814300,...","[1613048622.0, 1613048632.0, 1613048647.0, 161...",1714
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...","[97057, 644971, 947129, 58840, 640213, 954408,...","[1613048622.0, 1613048972.0, 1613049327.0, 161...",197
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816...","[929843, 816461, 926535, 112, 929843, 276471, ...","[1613048622.0, 1613048632.0, 1613048643.0, 161...",9296
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554...","[704814, 554780, 5737, 554780, 523251, 25538, ...","[1613048623.0, 1613083351.0, 1613083389.0, 161...",52
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144...","[919287, 144596, 626376, 919485, 144596, 18642...","[1613048623.0, 1613048673.0, 1613048722.0, 161...",268
...,...,...,...,...,...
672111,user_829403,"[[item_183824, 2021-05-12 12:59:10], [item_292...","[183824, 292633, 167515, 263005, 335217, 62884...","[1620824350.0, 1620824402.0, 1620824419.0, 162...",8
672112,user_829406,"[[item_795681, 2021-05-12 13:00:34], [item_358...","[795681, 358850]","[1620824434.0, 1620824576.0]",2
672113,user_829407,"[[item_211953, 2021-05-12 13:00:39], [item_183...","[211953, 183816]","[1620824439.0, 1620824447.0]",2
672114,user_829408,"[[item_485803, 2021-05-12 13:00:42], [item_104...","[485803, 1047]","[1620824442.0, 1620824517.0]",2


In [137]:
%%time
tail = 30
min_idx = 0
# 5개 전까지 확인
length =20
idx = 0
TIME_WEIGHT =1
parallel =1024
topn = 20
# for start_idx in range(len(grp_df)-5):
n = 200000  #chunk row size
list_df = [train_db[i:i+n] for i in range(0,train_db.shape[0],n)]
chunks_1 = list_df[0]
grp_chunk = chunks_1.groupby('user_id')
# chunk에서 user_id가 짤렸다면 unique user id 보다 길이가 더 늘어날 수 있으므로 len(df)+알파
# 161개의 청크이므로 중복은 161개 정도에서만 발생, 안 발생하도록 코딩하는 게 제일 좋지만
pairs = [{(0,0) : 0.0 for _ in range(0)} for _ in range(len(df)+300)]
total_len = len(df) 
max_idx = len(df)
curr_idx = 0

for idx in tqdm(range(0,max_idx, parallel)):
    row = df.iloc[idx:min(idx+parallel,max_idx)][['new_item_id','new_timestamp']].values
    temp =  np.array(list(map(lambda x : np.concatenate((np.reshape(x[0], newshape=(-1,1)) , np.reshape(x[1], newshape=(-1,1)) ),axis=1),row))) #1row에 1분 정도면 600분이면 10시간인데 그정도걸릴듯하다 d
    # 시간이 너무 오래걸리니 일단 넣고 numba 안에서 실행하면 빨라지는 지 확인
    # for input in temp:
    input = np.concatenate(temp)  
    
    # 다음 loop에서 꼬인다    
    if len(row) <= parallel:
      batch_len = len(row)
    else:
      batch_len = parallel
    for user_idx in range(batch_len):
      user_range =  len(temp[user_idx])
    # 1024개의 유저까지 indexing을 하고
    # 한 명 한명의 유저의 길이까지 짤라서 넘겨야한다
      get_pairs(idx ,input[curr_idx:user_range],cnts,max_idx,par_n)
      # print(len(input[curr_idx:user_range]))
      # curr_idx = user_range
      # print(user_idx)/
    print('test')

  0%|          | 0/657 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1714
197
9296
52
268
17
1328
59
23
18
3535
3
44
280
3006
48
59
14
5
8
129
370
22
18
954
60
1123
4535
590
911
10
131
223
118
7
17
269
73
3
246
859
35
662
2
17
3
22
849
152
75
2
29
2543
113
10
406
14
138
2
22
1554
16
87
6
15
58


  0%|          | 0/657 [00:19<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



66
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/execution.py", line 1310, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 37, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 319, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3

KeyboardInterrupt: ignored

In [133]:
idx

20

In [130]:
user_idx

10

array([[8.05696000e+05, 1.61304862e+09],
       [3.86903000e+05, 1.61304863e+09],
       [3.83200000e+03, 1.61304865e+09],
       ...,
       [7.84458000e+05, 1.62057312e+09],
       [6.82100000e+03, 1.62057317e+09],
       [9.44280000e+05, 1.62060035e+09]])

KeyError: ignored

In [ ]:
test_user_label = load_json('/gdrive/MyDrive/data/test_user_label.json')
test_user_seq_log = load_json('/gdrive/MyDrive/data/test_user_seq_log.json')
test_user_label_df = pd.DataFrame(list(test_user_label.items()), columns=['item_id', 'item_tiemstamp'])
test_user_seq_log = pd.DataFrame(list(test_user_seq_log.items()), columns=['item_id', 'item_timestamp'])
test_user_label_df.head()

In [48]:
test_user_seq_log = test_user_seq_log.rename(columns={'text':'item_timestamp'})

array([['item_944280', '2021-05-09 22:45:46'],
       ['item_6821', '2021-05-09 15:12:50'],
       ['item_784458', '2021-05-09 15:11:59'],
       ['item_888904', '2021-05-09 15:11:27'],
       ['item_369709', '2021-05-09 15:10:07'],
       ['item_784005', '2021-05-09 15:09:21'],
       ['item_453078', '2021-05-09 15:07:38'],
       ['item_396362', '2021-05-09 15:05:57'],
       ['item_760540', '2021-05-09 15:03:21'],
       ['item_171244', '2021-05-09 15:02:14'],
       ['item_156185', '2021-05-09 15:00:51'],
       ['item_99247', '2021-05-09 11:35:00'],
       ['item_6725', '2021-05-07 21:53:07'],
       ['item_18748', '2021-05-07 21:52:05'],
       ['item_228561', '2021-05-07 21:48:17'],
       ['item_580437', '2021-05-07 21:47:50'],
       ['item_773271', '2021-05-05 22:15:48'],
       ['item_580437', '2021-05-05 22:11:15'],
       ['item_65273', '2021-05-05 22:08:52'],
       ['item_766598', '2021-05-05 22:05:21']], dtype='<U19')

In [86]:
candidates = np.array(test_user_seq_log['item_timestamp'][0][::-1])

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자


NameError: ignored

In [ ]:
#test_cand = np.array(list(map( lambda x: np.array([np.int(x[0].split("_")[-1]),x[1]]) ,candidates)))
test_cand = np.array(list(map( lambda x: np.array([int(x[0].split("_")[-1]), int(time.mktime (datetime.strptime(x[1],"%Y-%m-%d %H:%M:%S").timetuple())) ]) ,candidates)))

In [54]:
candidates = np.array(test_user_seq_log['text'][0][::-1])

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자

In [111]:
test_cand

array([[    944280, 1620600346],
       [      6821, 1620573170],
       [    784458, 1620573119],
       [    888904, 1620573087],
       [    369709, 1620573007],
       [    784005, 1620572961],
       [    453078, 1620572858],
       [    396362, 1620572757],
       [    760540, 1620572601],
       [    171244, 1620572534],
       [    156185, 1620572451],
       [     99247, 1620560100],
       [      6725, 1620424387],
       [     18748, 1620424325],
       [    228561, 1620424097],
       [    580437, 1620424070],
       [    773271, 1620252948],
       [    580437, 1620252675],
       [     65273, 1620252532],
       [    766598, 1620252321]])

In [109]:
@nb.jit(nopython=True)
def test(candidates,topk,result,user_id):
  unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
  cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

  #Casting [unichr x 19] to int32 directly is unsupported.
  # for a in candidates:
    # t = nb.types.int32(a[0]) 
    # cnt[a[0]] = 0
  for a in candidates:
    # t = a[0].split("_")[-1]
    # t = int(a[0])
    unique_aids[a[0]] = 0
  print(unique_aids)  
  seq_weight = 0.5
  if len(unique_aids) >= 20:
    sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
    for a , w in zip(candidates,sequence_weight):     
      if a[0] not in cnt:
         cnt[a[0]] = 0
      cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
    result_candidates = heap_topk(cnt, 0, 20)
  else:
    result_candidates = list(unique_aids)
    for a in result_candidates:
      if a not in topk: continue
      for b in topk[a]:
        if b in unique_aids: continue
        if b not in cnt: cnt[b] = 0
        cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
    result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
  result[user_id] = np.array(result_candidates)
  

In [110]:

test(test_cand, topk,result_clicks ,user_id)

{944280: 0.0, 6821: 0.0, 784458: 0.0, 888904: 0.0, 369709: 0.0, 784005: 0.0, 453078: 0.0, 396362: 0.0, 760540: 0.0, 171244: 0.0, 156185: 0.0, 99247: 0.0, 6725: 0.0, 18748: 0.0, 228561: 0.0, 580437: 0.0, 773271: 0.0, 65273: 0.0, 766598: 0.0}


In [105]:

test(test_cand, topk,result_clicks ,user_id)

TypingError: ignored

In [62]:
test_user_seq_log.head()

,item_id,item_timestamp,new_item_id,new_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...",766598,1.620252e+09
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...",183462,1.619788e+09
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...",108368,1.620762e+09
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...",520146,1.615644e+09
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...",577279,1.620343e+09


In [148]:
test_user_seq_log['new_item_id'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ np.int64(val[0].split("_")[-1]) for val in x])  )

In [118]:
 )
test_user_seq_log['new_timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.array([ int(time.mktime(datetime.strptime(val[1],"%Y-%m-%d %H:%M:%S").timetuple())) for val in x ]) )

In [141]:
test_user_seq_log.head()

,user_id,item_timestamp,new_item_id,new_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294..."
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848..."
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680..."
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918..."
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357..."


In [83]:
test_user_seq_log = test_user_seq_log.rename(columns={'item_id':'user_id'})

In [122]:
test_user_seq_log[['user_id','new_item_id','new_timestamp']].values[:5]

array([['user_0', 766598,
        array([1620252321, 1620252532, 1620252675, 1620252948, 1620424070,
               1620424097, 1620424325, 1620424387, 1620560100, 1620572451,
               1620572534, 1620572601, 1620572757, 1620572858, 1620572961,
               1620573007, 1620573087, 1620573119, 1620573170, 1620600346])],
       ['user_1', 183462,
        array([1619788282, 1619788315, 1619788360, 1619788487, 1620050914,
               1620051029, 1620051132, 1620051180, 1620051291, 1620051314,
               1620132808, 1620132904, 1620133097, 1620221328, 1620307669,
               1620566935, 1620567001, 1620657125, 1620737680, 1620737741])],
       ['user_2', 108368,
        array([1620762088, 1620766788, 1620766797, 1620766801, 1620768498,
               1620768514, 1620768523, 1620768533, 1620768539, 1620768548,
               1620773193, 1620773213, 1620776036, 1620776050, 1620776057,
               1620776062, 1620810698, 1620811526, 1620811533, 1620811590])],
       ['user

In [ ]:
import heapq
# 캐싱해서 그런가
test(test_cand, topk,result_clicks ,user_id)

In [121]:
result_clicks['user_0']

array([944280,   6821, 784458, 888904, 369709, 784005, 453078, 396362,
       760540, 171244, 156185,  99247,   6725,  18748, 228561, 580437,
       773271,  65273, 766598])

In [120]:
test_user_seq_log['new_item_id'] = test_user_seq_log['item_timestamp'].apply(lambda x: np.int32(x[0][0].split("_")[-1]) )

In [53]:

test_user_seq_log['new_timestamp'] = test_user_seq_log['item_timestamp'].apply(lambda x: time.mktime(datetime.strptime(x[0][1],"%Y-%m-%d %H:%M:%S").timetuple()) )

In [93]:
#result place holder
result_clicks = nb.typed.Dict.empty(
    key_type = nb.types.string,
    value_type = nb.types.int64[:])
result_buy = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:]) 

In [ ]:

for idx in tqdm(range(0, len(test_user_seq_log), parallel)):
  row = test_user_seq_log.iloc[idx:min(idx+parallel , len(test_user_seq_log))][['user_id','new_item_id','new_timestamp']].values
  inference(row , result_clicks,topk)


In [124]:
test_cand

array([[    944280, 1620600346],
       [      6821, 1620573170],
       [    784458, 1620573119],
       [    888904, 1620573087],
       [    369709, 1620573007],
       [    784005, 1620572961],
       [    453078, 1620572858],
       [    396362, 1620572757],
       [    760540, 1620572601],
       [    171244, 1620572534],
       [    156185, 1620572451],
       [     99247, 1620560100],
       [      6725, 1620424387],
       [     18748, 1620424325],
       [    228561, 1620424097],
       [    580437, 1620424070],
       [    773271, 1620252948],
       [    580437, 1620252675],
       [     65273, 1620252532],
       [    766598, 1620252321]])

In [ ]:
@nb.jit(nopython=True)
def test(candidates,topk,result,user_id):
  unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
  cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

  #Casting [unichr x 19] to int32 directly is unsupported.
  # for a in candidates:
    # t = nb.types.int32(a[0]) 
    # cnt[a[0]] = 0
  for a in candidates:
    # t = a[0].split("_")[-1]
    # t = int(a[0])
    unique_aids[a[0]] = 0
  print(unique_aids)  
  seq_weight = 0.5
  if len(unique_aids) >= 20:
    sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
    for a , w in zip(candidates,sequence_weight):     
      if a[0] not in cnt:
         cnt[a[0]] = 0
      cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
    result_candidates = heap_topk(cnt, 0, 20)
  else:
    result_candidates = list(unique_aids)
    for a in result_candidates:
      if a not in topk: continue
      for b in topk[a]:
        if b in unique_aids: continue
        if b not in cnt: cnt[b] = 0
        cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
    result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
  result[user_id] = np.array(result_candidates)
  


In [157]:
@nb.jit(nopython=True, cache=True)
def inference(row, result_clicks,topk,seq_weight):
  for user_id, item_id, timestamp in row:
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
    cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

    # 최신순부터 거꾸로 배열
    new_item_id = item_id[::-1]
    new_timestamp = timestamp[::-1]

    for a in item_id:
    # t = a[0].split("_")[-1]
    # t = int(a[0])
      unique_aids[a] = 0
    seq_weight = 0.5
    
    if len(unique_aids) >= 20:
      sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(new_item_id)))[::-1]-1
      for a , w in zip(new_item_id,sequence_weight):
        if a not in cnt:
         cnt[a] = 0
        cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
      result_candidates = heap_topk(cnt, 0, 20)
    else:
      result_candidates = list(unique_aids)
      for a in result_candidates:
        if a not in topk: continue
        for b in topk[a]:
          if b in unique_aids: continue
          if b not in cnt: cnt[b] = 0
          cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
      result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
    result_clicks[user_id] = np.array(result_candidates)

In [129]:
# @nb.jit(nopython=True)
# def inference(row,result_clicks, topk):
#   inference_(row,result_clicks,topk, 0.1)

In [153]:
row = test_user_seq_log.iloc[idx:min(idx+parallel , len(test_user_seq_log))][['user_id','new_item_id','new_timestamp']].values

In [154]:
for user_id, item_id, timestamp in row:
  break

In [144]:

user_id

'user_0'

In [155]:

type(item_id[0])

numpy.int64

In [151]:

type(timestamp[0])

numpy.int64

In [149]:

test_user_seq_log.head()

,user_id,item_timestamp,new_item_id,new_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...","[766598, 65273, 580437, 773271, 580437, 228561...","[1620252321, 1620252532, 1620252675, 162025294..."
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...","[183462, 374145, 44563, 204302, 110301, 793635...","[1619788282, 1619788315, 1619788360, 161978848..."
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...","[108368, 41459, 667880, 869425, 932949, 869425...","[1620762088, 1620766788, 1620766797, 162076680..."
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...","[520146, 670721, 962828, 560874, 181147, 54449...","[1615643506, 1615935606, 1616058665, 161610918..."
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...","[577279, 374594, 99709, 382452, 747701, 717701...","[1620343383, 1620343410, 1620343540, 162034357..."


In [158]:

for idx in tqdm(range(0, len(test_user_seq_log), parallel)):
  row = test_user_seq_log.iloc[idx:min(idx+parallel , len(test_user_seq_log))][['user_id','new_item_id','new_timestamp']].values
  inference(row , result_clicks,topk,0.1)
  break

  0%|          | 0/574 [00:00<?, ?it/s]


TypingError: ignored

In [91]:
# @nb.jit(nopython=True)
# def test(candidates,topk,result,user_id):
#   unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
#   cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

#   #Casting [unichr x 19] to int32 directly is unsupported.
#   # for a in candidates:
#     # t = nb.types.int32(a[0]) 
#     # cnt[a[0]] = 0
#   for a in candidates:
#     # t = a[0].split("_")[-1]
#     # t = int(a[0])
#     unique_aids[a[0]] = 0
#   print(unique_aids)  
#   seq_weight = 0.5
#   if len(unique_aids) >= 20:
#     sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
#     for a , w in zip(candidates,sequence_weight):     
#       if a[0] not in cnt:
#          cnt[a[0]] = 0
#       cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
#     result_candidates = heap_topk(cnt, 0, 20)
#   else:
#     result_candidates = list(unique_aids)
#     for a in result_candidates:
#       if a not in topk: continue
#       for b in topk[a]:
#         if b in unique_aids: continue
#         if b not in cnt: cnt[b] = 0
#         cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
#     result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
#   result[user_id] = np.array(result_candidates)
  

In [95]:
user_id='user_0'

In [97]:
test_cand

array([[    944280, 1620600346],
       [      6821, 1620573170],
       [    784458, 1620573119],
       [    888904, 1620573087],
       [    369709, 1620573007],
       [    784005, 1620572961],
       [    453078, 1620572858],
       [    396362, 1620572757],
       [    760540, 1620572601],
       [    171244, 1620572534],
       [    156185, 1620572451],
       [     99247, 1620560100],
       [      6725, 1620424387],
       [     18748, 1620424325],
       [    228561, 1620424097],
       [    580437, 1620424070],
       [    773271, 1620252948],
       [    580437, 1620252675],
       [     65273, 1620252532],
       [    766598, 1620252321]])

In [119]:
import heapq
# 캐싱해서 그런가
test(test_cand, topk,result_clicks ,user_id)

{944280: 0.0, 6821: 0.0, 784458: 0.0, 888904: 0.0, 369709: 0.0, 784005: 0.0, 453078: 0.0, 396362: 0.0, 760540: 0.0, 171244: 0.0, 156185: 0.0, 99247: 0.0, 6725: 0.0, 18748: 0.0, 228561: 0.0, 580437: 0.0, 773271: 0.0, 65273: 0.0, 766598: 0.0}


In [54]:
test_user_seq_log

,item_id,item_timestamp,new_item_id,new_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652...",766598,1.620252e+09
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374...",183462,1.619788e+09
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414...",108368,1.620762e+09
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670...",520146,1.615644e+09
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374...",577279,1.620343e+09
...,...,...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810...",554650,1.620824e+09
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866...",310178,1.620824e+09
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492...",115233,1.620824e+09
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517...",503200,1.620824e+09


,item_id,text
0,user_0,"[item_223132, 2021-05-10 21:38:12]"
1,user_1,"[item_28643, 2021-05-11 21:12:24]"
2,user_2,"[item_77385, 2021-05-12 09:26:39]"
3,user_3,"[item_552518, 2021-04-07 23:02:18]"
4,user_4,"[item_421592, 2021-05-11 11:29:14]"


In [ ]:
# @nb.jit(nopython=True)
# def test(candidates,topk,result,user_id):
#   unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
#   cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

#   #Casting [unichr x 19] to int32 directly is unsupported.
#   # for a in candidates:
#     # t = nb.types.int32(a[0]) 
#     # cnt[a[0]] = 0
#   for a in candidates:
#     # t = a[0].split("_")[-1]
#     # t = int(a[0])
#     unique_aids[a[0]] = 0
#   print(unique_aids)  
#   seq_weight = 0.5
#   if len(unique_aids) >= 20:
#     sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
#     for a , w in zip(candidates,sequence_weight):     
#       if a[0] not in cnt:
#          cnt[a[0]] = 0
#       cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
#     result_candidates = heap_topk(cnt, 0, 20)
#   else:
#     result_candidates = list(unique_aids)
#     for a in result_candidates:
#       if a not in topk: continue
#       for b in topk[a]:
#         if b in unique_aids: continue
#         if b not in cnt: cnt[b] = 0
#         cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
#     result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
#   result[user_id] = np.array(result_candidates)
  

In [ ]:
candidates = np.array(test_user_seq_log['text'][0][::-1])

In [ ]:
candidates = np.array(test_user_seq_log['text'][0][::-1])

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자
test(test_cand, topk,result_clicks ,user_id)

In [45]:
a

NameError: ignored

In [44]:
df.head()

,user_id,item_timestamp,mktimestamp,item_timestamps_2,new_item_id,new_timestamp
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386...",1.613049e+09,"[805696.0, 1613048622.0]",805696,1.613049e+09
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449...",1.613049e+09,"[97057.0, 1613048622.0]",97057,1.613049e+09
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816...",1.613049e+09,"[929843.0, 1613048622.0]",929843,1.613049e+09
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554...",1.613049e+09,"[704814.0, 1613048623.0]",704814,1.613049e+09
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144...",1.613049e+09,"[919287.0, 1613048623.0]",919287,1.613049e+09


In [38]:
cnts.keys()

In [41]:
topk[468851]

KeyError: ignored

In [42]:
cnts[468851]

DictType[int64,float64]<iv=None>({446689: 5.0, 171464: 5.0, 241997: 5.0, 143143: 5.0, 89573: 3.0, 945910: 4.0, 38076: 5.0, 736701: 5.0, 424145: 4.0, 831745: 5.0, 944321: 3.0, 686802: 5.0, 31952: 5.0, 38040: 5.0, 447891: 5.0, 115956: 5.0, 625979: 5.0, 931351: 5.0, 922473: 5.0, 773029: 5.0, 92305: 3.0, 168649: 5.0, 392087: 5.0, 965347: 3.0, 628445: 5.0, 180738: 5.0, 733176: 5.0, 594772: 5.0, 503560: 5.0, 406829: 3.0, 121129: 5.0, 930780: 5.0, 496215: 5.0, 574693: 4.0, 910465: 4.0, 875293: 4.0, 124625: 4.0, 16536: 4.0, 248163: 3.0, 248696: 4.0, 733858: 4.0, 733992: 4.0, 126195: 4.0, 587996: 4.0, 194695: 4.0, 407699: 4.0, 789782: 3.0, 42042: 1.0, 646720: 4.0, 344186: 4.0, 8113: 4.0, 727459: 3.0, 149596: 4.0, 934114: 4.0, 548917: 4.0, 70023: 4.0, 35791: 3.0, 253787: 3.0, 779284: 3.0, 528557: 3.0, 812228: 3.0, 628393: 3.0, 562199: 3.0, 641557: 3.0, 884489: 3.0, 282760: 1.0, 836079: 3.0, 153429: 3.0, 283194: 3.0, 462172: 3.0, 244734: 3.0, 86183: 3.0, 785275: 3.0, 605732: 3.0, 259995: 3.0, 169

In [ ]:
get_topk(cnts, topk , topn)

In [ ]:

item_names_table = load_json('/gdrive/MyDrive/data/item_names_table.json')
item_df = pd.DataFrame(list(item_names_table.items()),columns = ['item_id','text'])
item_df['text_re'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in item_df['text']]
item_df.memory_usage()

Index          128
item_id    7761960
text       7761960
text_re    7761960
dtype: int64

In [ ]:
item_timestamps = np.array(grp_df[['item_id','mktimestamp']].values)

In [ ]:
cnts

In [ ]:
test_user_label = load_json('../data/test_user_label.json')
test_user_seq_log = load_json('../data/test_user_seq_log.json')
test_user_label_df = pd.DataFrame(list(test_user_label.items()), columns=['item_id', 'text'])
test_user_seq_log = pd.DataFrame(list(test_user_seq_log.items()), columns=['item_id', 'text'])
test_user_label_df.head()

In [ ]:
np.linspace(0.1 , 1 , 20)

array([0.1       , 0.14736842, 0.19473684, 0.24210526, 0.28947368,
       0.33684211, 0.38421053, 0.43157895, 0.47894737, 0.52631579,
       0.57368421, 0.62105263, 0.66842105, 0.71578947, 0.76315789,
       0.81052632, 0.85789474, 0.90526316, 0.95263158, 1.        ])

In [ ]:
# sequence_weight
np.power(2, np.linspace(0.1 , 1 , 20))[::-1] -1

array([1.        , 0.93539975, 0.87288609, 0.81239163, 0.75385116,
       0.69720154, 0.64238172, 0.58933258, 0.53799694, 0.48831944,
       0.44024654, 0.39372639, 0.34870885, 0.30514539, 0.26298903,
       0.22219432, 0.18271729, 0.14451537, 0.10754738, 0.07177346])

In [ ]:
#cnt 에 sequence weight 를 더하고 heap_topk로 뽑는다

In [ ]:
#그리고 result[user_id] 에 result_candidates를 더하는데 다른 점이뭐지 
# test ops weight와 sequence weight 를 기준으로 그 유저가 구매한 것중에서 추천하는 거 같은데
#후보의 길이가 20이 넘지 않으면 topks 로 추천하고
# 

In [ ]:
#aids : 전체 aids
#row : df의 row 

In [ ]:
row =  df.iloc[idx:min(idx+parallel, len(df))][['user_id','item_timestamp']].values

In [ ]:
for user_id , item_timestamp in row:
  break

In [ ]:
# Opening JSON file
f = open(DATA_DIR + 'test_user_seq_log.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Iterating through the jsoa

In [ ]:
# 20개에 검색에 대해서 마지막 검색을 예측 해야함
test_user_seq_log.head()

,item_id,text
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652..."
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374..."
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414..."
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670..."
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374..."


In [ ]:
candidates = np.array(test_user_seq_log['text'][0][::-1])

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자
test(test_cand, topk,result_clicks ,user_id)

In [ ]:
#result place holder
result_clicks = nb.typed.Dict.empty(
    key_type = nb.types.string,
    value_type = nb.types.int64[:])
result_buy = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:]) 

In [ ]:

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자
test(test_cand, topk,result_clicks ,user_id)

In [ ]:
# @nb.jit(nopython=True)
# def test(candidates,topk,result,user_id):
#   unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type= nb.types.float64) 
#   cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)

#   #Casting [unichr x 19] to int32 directly is unsupported.
#   # for a in candidates:
#     # t = nb.types.int32(a[0]) 
#     # cnt[a[0]] = 0
#   for a in candidates:
#     # t = a[0].split("_")[-1]
#     # t = int(a[0])
#     unique_aids[a[0]] = 0
#   print(unique_aids)  
#   seq_weight = 0.5
#   if len(unique_aids) >= 20:
#     sequence_weight = np.power(2 , np.linspace(seq_weight, 1, len(candidates)))[::-1]-1
#     for a , w in zip(candidates,sequence_weight):     
#       if a[0] not in cnt:
#          cnt[a[0]] = 0
#       cnt[a[0]] += w # * 클릭 ,장바구니, 구매에 따른 가중치 추가가능
#     result_candidates = heap_topk(cnt, 0, 20)
#   else:
#     result_candidates = list(unique_aids)
#     for a in result_candidates:
#       if a not in topk: continue
#       for b in topk[a]:
#         if b in unique_aids: continue
#         if b not in cnt: cnt[b] = 0
#         cnt[b] += 1 # 정렬된 topk에서 co-visitation이 높은 아이템을 순서대로 가져와 모자란 길이만큼 채운다
#     result_candidates.extend(heap_topk(cnt, 0, 20 - len(result_candidates)))
#   result[user_id] = np.array(result_candidates)
  

In [ ]:
nb.types.int32(3)

3

In [ ]:
# 추천이 나오긴 했는데 흐으으음
result_clicks['user_0']

array([944280,   6821, 784458, 888904, 369709, 784005, 453078, 396362,
       760540, 171244, 156185,  99247,   6725,  18748, 228561, 580437,
       773271,  65273, 766598, 847734])

In [ ]:
result_list = list(map(lambda x: 'item_'+str(x),result_clicks['user_0']))

In [ ]:
result_clicks['user_1']

In [ ]:
test_user_label = load_json('/gdrive/MyDrive/data/test_user_label.json')
test_user_label_df = pd.DataFrame(list(test_user_label.items()), columns=['item_id', 'text'])
test_user_label_df.head()

,item_id,text
0,user_0,"[item_223132, 2021-05-10 21:38:12]"
1,user_1,"[item_28643, 2021-05-11 21:12:24]"
2,user_2,"[item_77385, 2021-05-12 09:26:39]"
3,user_3,"[item_552518, 2021-04-07 23:02:18]"
4,user_4,"[item_421592, 2021-05-11 11:29:14]"


In [ ]:
test_user_seq_log

,user_id,item_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652..."
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374..."
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414..."
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670..."
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374..."
...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810..."
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866..."
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492..."
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517..."


In [ ]:
item_df[item_df.item_id.isin(result_list)]

,item_id,text,text_re
6725,item_6725,비비안 스킨핏 백화점 동일 원단 FREE FIT 브라팬티세트,비비안 스킨핏 백화점 동일 원단 FREE FIT 브라팬티세트
6821,item_6821,[엠포리오 아르마니] EA7 네오 클래식 스니커즈 (여),엠포리오 아르마니 EA7 네오 클래식 스니커즈 여
18748,item_18748,SEKANSKEEN 프리미엄,SEKANSKEEN 프리미엄
65273,item_65273,"[현대5%할인,일시불 할인]버버리 트렌톤 빅백(8019899,8019900)",현대5 할인 일시불 할인 버버리 트렌톤 빅백 8019899 8019900
99247,item_99247,BURBERRY 애쉬비 캔버스 투웨이 호보백 (3945726 1/ 4059749 1...,BURBERRY 애쉬비 캔버스 투웨이 호보백 3945726 1 4059749 1...
156185,item_156185,이상봉 골프 21SS 시그니처 웨더자켓 1종 [여성],이상봉 골프 21SS 시그니처 웨더자켓 1종 여성
171244,item_171244,[미리주문10%][바스키아X원빈]21SUMMER트리코트골프팬츠3종 여성,미리주문10 바스키아X원빈 21SUMMER트리코트골프팬츠3종 여성
228561,item_228561,마크제이콥스+트레블러+토트백(M0016161),마크제이콥스 트레블러 토트백 M0016161
369709,item_369709,LG DIOS 냉장고 F873SS31H 5도어 매직스페이스,LG DIOS 냉장고 F873SS31H 5도어 매직스페이스
396362,item_396362,라삐아프 린넨 팬츠 3종,라삐아프 린넨 팬츠 3종


In [ ]:
item_df

,item_id,text,text_re
0,item_0,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET,DA 남자 빅사이즈 절개 맨투맨 절개 트레이닝팬츠 SET
1,item_1,텐피엠 [카시오] MTP-V005L 남성가죽시계+ 블랙 나토밴드.,텐피엠 카시오 MTP V005L 남성가죽시계 블랙 나토밴드
2,item_2,[바보사랑]여성 옥스퍼드 로퍼-(주)태블리스,바보사랑 여성 옥스퍼드 로퍼 주 태블리스
3,item_3,[해외][블랙모어스] 리프리놀/ 초록홍합 100캡슐 X 3,해외 블랙모어스 리프리놀 초록홍합 100캡슐 X 3
4,item_4,[백화점 SAY][크로커다일]경량덕 다운 점퍼(CL9WDW901),백화점 SAY 크로커다일 경량덕 다운 점퍼 CL9WDW901
...,...,...,...
970240,item_970240,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페,접이식 벤치의자 스툴 스탠드 의자 홈바 인테리어 카페
970241,item_970241,[오너클랜]기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장,오너클랜 기획 신발정리대 소형 6단 1P 현관신발장 오픈신발장
970242,item_970242,[피우다][피우다 (PIUDA)] 여름바다 상자 3p 세트 (수납함/수납,피우다 피우다 PIUDA 여름바다 상자 3p 세트 수납함 수납
970243,item_970243,[지큐][지큐/쿨제이][빅사이즈 추가] 모자이크 롱 숄 가디건,지큐 지큐 쿨제이 빅사이즈 추가 모자이크 롱 숄 가디건


In [ ]:
df[df.user_id == 'user_0']

,user_id,item_timestamp
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386..."


In [ ]:

# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우
# 드디어 casting, array에 서로 다른 타입을 담을 수 없기 때문에 둘 다변환해서 넘기자
test(test_cand, topk,result_clicks ,user_id)

{944280: 0.0, 6821: 0.0, 784458: 0.0, 888904: 0.0, 369709: 0.0, 784005: 0.0, 453078: 0.0, 396362: 0.0, 760540: 0.0, 171244: 0.0, 156185: 0.0, 99247: 0.0, 6725: 0.0, 18748: 0.0, 228561: 0.0, 580437: 0.0, 773271: 0.0, 65273: 0.0, 766598: 0.0}


In [88]:

test_cand = np.array(list(map( lambda x: np.array([int(x[0].split("_")[-1]), int(time.mktime (datetime.strptime(x[1],"%Y-%m-%d %H:%M:%S").timetuple())) ]) ,candidates)))

array([[    944280, 1620600346],
       [      6821, 1620573170],
       [    784458, 1620573119],
       [    888904, 1620573087],
       [    369709, 1620573007],
       [    784005, 1620572961],
       [    453078, 1620572858],
       [    396362, 1620572757],
       [    760540, 1620572601],
       [    171244, 1620572534],
       [    156185, 1620572451],
       [     99247, 1620560100],
       [      6725, 1620424387],
       [     18748, 1620424325],
       [    228561, 1620424097],
       [    580437, 1620424070],
       [    773271, 1620252948],
       [    580437, 1620252675],
       [     65273, 1620252532],
       [    766598, 1620252321]])

In [ ]:
type(test_cand[0][1])

numpy.int64

numpy.str_

In [ ]:
for idx in range(len(test_cand)):
  test_cand[idx][0] = np.float32(test_cand[idx][0])

In [ ]:

type(test_cand[0][0])

numpy.str_

In [ ]:
test_list = nb.typed.List[nb.types.int32]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


int

In [ ]:
t=  np.array(list(map(lambda x: [np.int(x[0]),x[1]], test_cand)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [ ]:
# map을 써도 무조건 str로 된다 .. 하
type(t[0][0])

numpy.str_

In [ ]:
test_cand= list(list(map(lambda x: [np.int(x[0]),x[1]], test_cand)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [ ]:
# 캐싱해서 그런가
# int 를 넘겨야 되네 밖에서 왜 시발 개같은 경우

test(test_cand[0][0], topk,result_clicks ,user_id)

In [ ]:
for a in test_cand:
  print(type(a[0]))
  break

<class 'int'>


In [ ]:
for a in test_cand:
    # t = nb.types.int32(a[0])
    # print(type(t))
    print(a[0])

944280
6821
784458
888904
369709
784005
453078
396362
760540
171244
156185
99247
6725
18748
228561
580437
773271
580437
65273
766598


TypingError: ignored

NameError: ignored

TypingError: ignored

In [ ]:
user_id = test_user_seq_log['user_id'][0]

TypingError: ignored

In [ ]:
@nb.jit(nopython=True, cache=True)
def test_2( a):
  cnt = nb.typed.Dict.empty(key_type= nb.types.int64, value_type = nb.types.float64)
  cnt[a] = 0
 

In [ ]:

test_2(1)

In [ ]:
 # refleceted list는 사용할 수 없다

test(candidates,topk,result_clicks ,user_id)

In [ ]:
test_cand = np.array(list(map( lambda x: np.array([np.int(x[0].split("_")[-1]),x[1]]) ,candidates)))
test_cand = np.array(list(map( lambda x: np.array([int(x[0].split("_")[-1]), int(time.mktime (datetime.strptime(x[1],"%Y-%m-%d %H:%M:%S").timetuple())) ]) ,candidates)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [ ]:
test_cand = np.array(list(map( lambda x: np.array([int(x[0].split("_")[-1]), int(time.mktime (datetime.strptime(x[1],"%Y-%m-%d %H:%M:%S").timetuple())) ]) ,candidates)))

In [104]:
import numpy as np
import numba as nb
 
nb_list = nb.typed.List

In [ ]:
nb_candidates = nb.typed.List(candidates)

TypingError: ignored

TypingError: ignored

In [ ]:
for a in candidates:
  t = int(a[0].split("_")[-1])

766598

TypingError: ignored

In [ ]:
def generate_train(df):
    user_ids = []
    item_ids = []
    timestamps = []
    for rows in tqdm(df.iterrows()):
        item_timestamp = rows[1]['item_timestamp']
        user_id = rows[1]['user_id']
        for idx in range(len(item_timestamp)):
            item = item_timestamp[idx][0]
            timestamp = item_timestamp[idx][1]
            user_ids.append(user_id)
            item_ids.append(item)
            timestamps.append(timestamp)
    train_df = pd.DataFrame(user_ids,columns=['user_ids'])
    train_df['item_ids'] = item_ids
    train_df['timestamp'] = timestamps
    return train_df

In [ ]:
test_user_seq_log = test_user_seq_log.rename(columns={'text':'item_timestamp'})

In [ ]:
test_user_seq_log = test_user_seq_log.rename(columns={'item_id':'user_id'})

In [ ]:
type(candidates[0][0].split("_")[-1])

str

In [ ]:
test_user_seq_log

,user_id,item_timestamp
0,user_0,"[[item_766598, 2021-05-05 22:05:21], [item_652..."
1,user_1,"[[item_183462, 2021-04-30 13:11:22], [item_374..."
2,user_2,"[[item_108368, 2021-05-11 19:41:28], [item_414..."
3,user_3,"[[item_520146, 2021-03-13 13:51:46], [item_670..."
4,user_4,"[[item_577279, 2021-05-06 23:23:03], [item_374..."
...,...,...
587481,user_829386,"[[item_554650, 2021-05-12 12:55:35], [item_810..."
587482,user_829390,"[[item_310178, 2021-05-12 12:56:18], [item_866..."
587483,user_829393,"[[item_115233, 2021-05-12 12:56:42], [item_492..."
587484,user_829397,"[[item_503200, 2021-05-12 12:58:08], [item_517..."


In [ ]:
for a in candidates:
  break

In [40]:
test(candidates)

NameError: ignored

In [ ]:
topk[944280]

In [116]:
type(test_cand[0])

numpy.ndarray

In [107]:
test(test_cand, topk,result_clicks ,user_id)


{944280: 0.0, 6821: 0.0, 784458: 0.0, 888904: 0.0, 369709: 0.0, 784005: 0.0, 453078: 0.0, 396362: 0.0, 760540: 0.0, 171244: 0.0, 156185: 0.0, 99247: 0.0, 6725: 0.0, 18748: 0.0, 228561: 0.0, 580437: 0.0, 773271: 0.0, 65273: 0.0, 766598: 0.0}


In [ ]:
@nb.jit(nopython=True, cache=True)
def inference_(aids, row):
    for session, idx, length in row:
        unique_aids = nb.typed.Dict.empty(key_tpye= nb.types.int64 , value_type = nb.types.float64)
        cnt = nb.typed.Dict.empty(key_type = nb.types.int64 , value_type = nb.types.float64)

        candidates = aids[idx:idx+length][::-1]
        candidates_ops = ops[idx:idx+length][::-1]

        for a in candidates:
            unique_aids[a] = 0

        if len(unique_aids) >= 20:
            sequence_weight = np.power(2, np.linspace(seq_weight, 1, len(candidates)))[::-1] -1
            for a , op ,w in zip(candidates, candidates_ops , sequence_weight)
                if a not in cnt: cnt[a] = 0
                cnt[a] += 2 * test_ops_weights[op]
            result_candidates = heap_topk(cnt,0,20)

        else:
            result_candidates = list(unique_aids)
            for a in result_candidates:
                if a not in topk: continue
                for b in topk[a]:
                    if b in unique_aids: continue
                    if b not in cnt: cnt[b] = 0
                    cnt[b] += 1
            result_candidates.extend(heap_topk(cnt, 0 , 20 -len(result_candidates)))
        result[session] = np.array(result_candidates)
      
@nb.jit(nopython=True)
def inference_(aids,):
  inference_(aids, ops,row ,result_clicks, topk_clicks, test_ops_weights,0.1)
  inference_(aids, ops, row, result_buy, topk_buy , test_ops_weights, 0.5)


In [ ]:
df.head()

,user_id,item_timestamp
0,user_0,"[[item_805696, 2021-02-11 13:03:42], [item_386..."
1,user_1,"[[item_97057, 2021-02-11 13:03:42], [item_6449..."
2,user_2,"[[item_929843, 2021-02-11 13:03:42], [item_816..."
3,user_3,"[[item_704814, 2021-02-11 13:03:43], [item_554..."
4,user_4,"[[item_919287, 2021-02-11 13:03:43], [item_144..."


In [ ]:
parallel = 1024

In [ ]:

idx =0 

array([['user_0',
        list([['item_805696', '2021-02-11 13:03:42'], ['item_386903', '2021-02-11 13:03:52'], ['item_3832', '2021-02-11 13:04:07'], ['item_849824', '2021-02-11 13:05:04'], ['item_815594', '2021-02-11 13:06:19'], ['item_814300', '2021-02-11 13:07:28'], ['item_930329', '2021-02-11 13:09:14'], ['item_823097', '2021-02-11 13:09:40'], ['item_837711', '2021-02-11 13:13:35'], ['item_704626', '2021-02-11 13:15:10'], ['item_46095', '2021-02-11 13:17:02'], ['item_542895', '2021-02-11 13:20:31'], ['item_229908', '2021-02-11 13:23:45'], ['item_935600', '2021-02-11 13:25:07'], ['item_700459', '2021-02-11 13:25:53'], ['item_114309', '2021-02-11 13:26:51'], ['item_212637', '2021-02-11 21:24:05'], ['item_691869', '2021-02-12 00:38:57'], ['item_648359', '2021-02-12 00:39:12'], ['item_202047', '2021-02-12 07:43:52'], ['item_929659', '2021-02-12 07:44:14'], ['item_704626', '2021-02-12 07:44:54'], ['item_930329', '2021-02-12 07:44:59'], ['item_190207', '2021-02-12 07:48:01'], ['item_6265

In [ ]:

for idx in tqdm(range(len(df) - len(df_test) , len(df) , parallel)):
  row = df.iloc[idx:min(idx+parallel, len(df))][['session','idx','length']].values
  inference(aids, ops , row , result_clicks , result_buy , topks[TIME_WEIGHT] , topks[OP_WEIGHT], test_ops_weight)

In [ ]:
#추천 방식

#

In [ ]:
import numpy as np


def ndcg(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: ndcg@k
    :return: ndcg@k score
    '''

    ndcg_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    ndcg_ += 1 / np.log2(idx + 2)
                break

    return float(ndcg_ / len(recs))


def hit(recs: dict, k: int):
    '''
    :param recs: reco result and label
        example:
            {user: {'recs': [reco_item1, ..., reco_item@], 'gt': gt_item(== label)}}
    :param k: hit@k
    :return: hit@k(==recall@k) score
    '''

    hit_ = 0
    for user, each in recs.items():
        reco_items = each['recs']
        gt_item = each['gt']

        for idx, reco_item in enumerate(reco_items):
            if gt_item == reco_item:
                if idx < k:
                    hit_ += 1
                break

    return float(hit_ / len(recs))
